In [ ]:
import codecs

from bs4 import BeautifulSoup

from konlpy.tag import Okt

import urllib.request

import os, re, json, random

In [8]:
# 마르코프 체인 딕셔너리 만들기 --- (※1)

def make_dic(words):

    tmp = ["@"]

    dic = {}

    for word in words:

        tmp.append(word)

        if len(tmp) < 3: continue

        if len(tmp) > 3: tmp = tmp[1:]

        set_word3(dic, tmp)

        if word == ".":

            tmp = ["@"]

            continue

    return dic

# 딕셔너리에 데이터 등록하기 --- (※2)

def set_word3(dic, s3):

    w1, w2, w3 = s3

    if not w1 in dic: dic[w1] = {}

    if not w2 in dic[w1]: dic[w1][w2] = {}

    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0

    dic[w1][w2][w3] += 1

# 문장 만들기 --- (※3)

def make_sentence(dic):

    ret = []

    if not "@" in dic: return "no dic" 

    top = dic["@"]

    w1 = word_choice(top)

    w2 = word_choice(top[w1])

    ret.append(w1)

    ret.append(w2)

    while True:

        w3 = word_choice(dic[w1][w2])

        ret.append(w3)

        if w3 == ".": break

        w1, w2 = w2, w3

    ret = "".join(ret)

    # 띄어쓰기

    params = urllib.parse.urlencode({

        "_callback": "",

        "q": ret

    })


    # 리턴

    return ret

def spell(s):

    import requests

    data={}

    data["sentence"]=s.encode("utf-8")

    r = requests.post("https://alldic.daum.net/grammar_checker.do", data=data)

    soup = BeautifulSoup(r.text,'html.parser')

    result=[]

    for text in soup.find_all('a'):

        if not text.get('data-error-output') == None:

            a=text.get('data-error-output')

            result.append(a)

    return(result)


def word_choice(sel):

    keys = sel.keys()

    return random.choice(list(keys))



#---main---
# 문장 읽어 들이기 --- (※4)

toji_file = "toji.txt"

dict_file = "markov-toji.json"

if not os.path.exists(dict_file):

    # 토지 텍스트 파일 읽어 들이기

    fp = codecs.open("BEXX0014.txt", "r", encoding="utf-16")

    soup = BeautifulSoup(fp, "html.parser")

    body = soup.select_one("body")

    text = body.getText()

    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결

    # 형태소 분석

    twitter = Okt()

    malist = twitter.pos(text, norm=True)

    words = []

    for word in malist:

        # 구두점 등은 대상에서 제외(단 마침표는 포함)

        if not word[1] in ["Punctuation"]:

            words.append(word[0])

        if word[0] == ".":

            words.append(word[0])

    # 딕셔너리 생성

    dic = make_dic(words)

    json.dump(dic, open(dict_file,"w", encoding="utf-8"))

else:

    dic = json.load(open(dict_file,"r"))

# 문장 만들기 --- (※6)

for i in range(3):

    s = make_sentence(dic)

    print(s)
    print(spell(s))
    print("---")

장지에서얼쩡거리는것을감추려하지는않았으나자기감정에대한명칭이있지
서말했다.
['장지에서 얼쩡거리는 것을 감추려 하지는 않았으나 자기감정에 대한 명칭이 있지', '서말 했다.']
---
팔라는말이마음은고향에데리고간곳없고무시무시한살기를내어뿜고있었다.
['팔라는 말이 마음은 고향에 데리고 간 곳 없고 무시무시한 살기를내어뿜고있었다.']
---
병수얼굴을붉힌다.
['병수 얼굴을 붉힌다.']
---
